In [6]:
import cv2
import numpy as np
from datetime import datetime
import time
from collections import deque
from ultralytics import YOLO

In [7]:
def calculate_angle(a, b, c):
    """Calculate angle between three points"""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

def extract_yolo_skeleton(results):
    """Extract skeleton from YOLO pose detection"""
    if results[0].keypoints is None or len(results[0].keypoints.data) == 0:
        return None
    
    # Get keypoints (17 COCO keypoints)
    keypoints = results[0].keypoints.data[0].cpu().numpy()
    
    skeleton = {}
    
    # YOLO COCO keypoint mapping (17 points)
    # 0: nose, 1-2: eyes, 3-4: ears, 5: left_shoulder, 6: right_shoulder,
    # 7: left_elbow, 8: right_elbow, 9: left_wrist, 10: right_wrist,
    # 11: left_hip, 12: right_hip, 13: left_knee, 14: right_knee,
    # 15: left_ankle, 16: right_ankle
    
    # Only use points with confidence > 0.5
    def get_point(idx):
        if idx < len(keypoints) and keypoints[idx][2] > 0.5:
            return (int(keypoints[idx][0]), int(keypoints[idx][1]))
        return None
    
    # Map to our skeleton format
    if get_point(0): skeleton['head'] = get_point(0)
    if get_point(0): skeleton['neck'] = get_point(0)  # Approximate
    if get_point(5): skeleton['left_shoulder'] = get_point(5)
    if get_point(6): skeleton['right_shoulder'] = get_point(6)
    if get_point(7): skeleton['left_elbow'] = get_point(7)
    if get_point(8): skeleton['right_elbow'] = get_point(8)
    if get_point(9): skeleton['left_hand'] = get_point(9)
    if get_point(10): skeleton['right_hand'] = get_point(10)
    if get_point(11): skeleton['left_hip'] = get_point(11)
    if get_point(12): skeleton['right_hip'] = get_point(12)
    if get_point(13): skeleton['left_knee'] = get_point(13)
    if get_point(14): skeleton['right_knee'] = get_point(14)
    if get_point(15): skeleton['left_foot'] = get_point(15)
    if get_point(16): skeleton['right_foot'] = get_point(16)
    
    # Calculate torso center if shoulders and hips available
    if all(k in skeleton for k in ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']):
        torso_x = (skeleton['left_shoulder'][0] + skeleton['right_shoulder'][0] + 
                   skeleton['left_hip'][0] + skeleton['right_hip'][0]) // 4
        torso_y = (skeleton['left_shoulder'][1] + skeleton['right_shoulder'][1] + 
                   skeleton['left_hip'][1] + skeleton['right_hip'][1]) // 4
        skeleton['torso'] = (torso_x, torso_y)
    
    return skeleton if len(skeleton) > 0 else None

In [8]:
def draw_skeleton(frame, skeleton):
    """Draw skeleton connections on frame"""
    if skeleton is None:
        return frame
    
    # Define skeleton connections
    connections = [
        ('head', 'neck'),
        ('neck', 'left_shoulder'),
        ('neck', 'right_shoulder'),
        ('left_shoulder', 'left_elbow'),
        ('left_elbow', 'left_hand'),
        ('right_shoulder', 'right_elbow'),
        ('right_elbow', 'right_hand'),
        ('neck', 'torso'),
        ('torso', 'left_hip'),
        ('torso', 'right_hip'),
        ('left_hip', 'right_hip'),
        ('left_hip', 'left_knee'),
        ('left_knee', 'left_foot'),
        ('right_hip', 'right_knee'),
        ('right_knee', 'right_foot'),
    ]
    
    # Color coding
    colors = {
        'head': (255, 255, 0),      # Yellow
        'neck': (255, 255, 0),
        'torso': (255, 0, 0),        # Blue
        'left_shoulder': (255, 0, 0),
        'right_shoulder': (255, 0, 0),
        'left_hip': (255, 0, 0),
        'right_hip': (255, 0, 0),
        'left_elbow': (0, 255, 0),   # Green (arms)
        'right_elbow': (0, 255, 0),
        'left_hand': (0, 255, 0),
        'right_hand': (0, 255, 0),
        'left_knee': (255, 0, 255),  # Magenta (legs)
        'right_knee': (255, 0, 255),
        'left_foot': (255, 0, 255),
        'right_foot': (255, 0, 255),
    }
    
    # Draw connections
    for start_joint, end_joint in connections:
        if start_joint in skeleton and end_joint in skeleton:
            cv2.line(frame, skeleton[start_joint], skeleton[end_joint], (0, 255, 255), 3)
    
    # Draw joints
    for joint_name, joint_pos in skeleton.items():
        color = colors.get(joint_name, (255, 255, 255))
        cv2.circle(frame, joint_pos, 6, color, -1)
        cv2.circle(frame, joint_pos, 7, (0, 0, 0), 1)
    
    return frame

In [9]:
def body_pose_tracker(camera_id=0, show_contour=True):
    """
    Real-time body pose tracking using YOLOv8 Pose
    
    Args:
        camera_id: Camera index (default: 0)
        show_contour: Show body contour (not used)
    
    Controls:
        Q - Quit
        S - Save frame
        C - Toggle YOLO keypoints visibility
    """
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print("Error: Could not open camera")
        return
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    # Load YOLOv8 Pose model
    print("Loading YOLOv8 Pose model...")
    model = YOLO('yolov8n-pose.pt')  # Nano model for speed
    
    prev_time = 0
    show_yolo_keypoints = True
    
    print("Starting body pose tracker with YOLOv8...")
    print("Ready to track!")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame")
            break
        
        curr_time = time.time()
        fps = 1 / (curr_time - prev_time) if prev_time > 0 else 0
        prev_time = curr_time
        
        # Run YOLO pose detection
        results = model(frame, verbose=False)
        
        # Extract skeleton
        skeleton = extract_yolo_skeleton(results)
        
        # Draw YOLO keypoints if enabled
        if show_yolo_keypoints and results[0].keypoints is not None:
            frame = results[0].plot()
        
        # Draw custom skeleton overlay
        if skeleton:
            frame = draw_skeleton(frame, skeleton)
        
        # Display info
        cv2.putText(frame, f'FPS: {fps:.1f}', (20, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        status = "Tracking" if skeleton else "No body detected"
        status_color = (0, 255, 0) if skeleton else (0, 165, 255)
        cv2.putText(frame, status, (20, 75),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, status_color, 2)
        
        cv2.putText(frame, "Q:Quit | S:Save | C:Toggle YOLO Keypoints", 
                   (10, frame.shape[0] - 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        cv2.imshow('Body Pose Tracker - YOLOv8', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key in (ord('q'), ord('Q')):
            break
        elif key in (ord('s'), ord('S')):
            img_name = f'pose_{datetime.now().strftime("%Y%m%d_%H%M%S")}.jpg'
            cv2.imwrite(img_name, frame)
            print(f"Saved: {img_name}")
        elif key in (ord('c'), ord('C')):
            show_yolo_keypoints = not show_yolo_keypoints
    
    cap.release()
    cv2.destroyAllWindows()
    print("Tracker stopped")

In [10]:
body_pose_tracker(camera_id=1, show_contour=True)

Loading YOLOv8 Pose model...
Starting body pose tracker with YOLOv8...
Ready to track!
Tracker stopped
